In [1]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
import glob
import json
file_dir='/global/cfs/cdirs/dune/users/mkramer/mywork/2x2_sim/run-convert2h5/output/MiniRun3_1E19_RHC.convert2h5_v3/EDEPSIM_H5/'

In [2]:
def get_unique_spills(sim_h5):
    return np.unique(sim_h5['trajectories']['eventID'])

In [3]:
def get_spill_data(sim_h5, spill_id):
    ghdr_spill_mask = sim_h5['genie_hdr'][:]['eventID']==spill_id
    gstack_spill_mask = sim_h5['genie_stack'][:]['eventID']==spill_id
    traj_spill_mask = sim_h5['trajectories'][:]['eventID']==spill_id
    vert_spill_mask = sim_h5['vertices'][:]['eventID']==spill_id
    seg_spill_mask = sim_h5['segments'][:]['eventID']==spill_id
    
    ghdr = sim_h5['genie_hdr'][ghdr_spill_mask]
    gstack = sim_h5['genie_stack'][gstack_spill_mask]
    traj = sim_h5['trajectories'][traj_spill_mask]
    vert = sim_h5['vertices'][vert_spill_mask]
    seg = sim_h5['segments'][seg_spill_mask]
    
    return ghdr, gstack, traj, vert, seg

In [4]:
def tpc_bounds(i):
    active_tpc_widths=[30.6, 130., 64.] # cm
    tpcs_relative_to_module=[[-15.7,0.,0.],[15.7,0.,0.]]
    modules_relative_to_2x2=[[-33.5,0.,-33.5],
                            [33.5,0.,-33.5],
                            [-33.5,0.,33.5],
                            [33.5,0.,33.5]]
    detector_center=[0.,52.25,0.]
    tpc_bounds=np.array([-active_tpc_widths[i]/2., active_tpc_widths[i]/2.])
    tpc_bounds_relative_to_2x2=[]
    for tpc in tpcs_relative_to_module:
        tpc_bound_relative_to_module = tpc_bounds + tpc[i]
        for module in modules_relative_to_2x2:
            bound = tpc_bound_relative_to_module + module[i]
            tpc_bounds_relative_to_2x2.append(bound)
    bounds_relative_to_NDhall = np.array(tpc_bounds_relative_to_2x2) + detector_center[i]
    return np.unique(bounds_relative_to_NDhall, axis=0)

In [5]:
def fiducialized_vertex(vert_pos):
    flag=False; x_drift_flag=False; y_vertical_flag=False; z_beam_flag=False
    for i in range(3):
        for i_bounds, bounds in enumerate(tpc_bounds(i)):
            if vert_pos[i]>bounds[0] and vert_pos[i]<bounds[1]:
                if i==0: x_drift_flag=True; break
                if i==1: y_vertical_flag=True
                if i==2: z_beam_flag=True
    if x_drift_flag==True and y_vertical_flag==True and z_beam_flag==True:
        flag=True
    return flag

In [6]:
def total_edep_charged_e(trackID, traj, seg):
    seg_id_mask=seg['trackID']==trackID
    total_e=0.
    for sg in seg[seg_id_mask]: total_e+=sg['dE']
    return total_e

In [7]:
def total_edep_length(trackID, traj, seg):
    seg_id_mask=seg['trackID']==trackID
    length=0.
    for sg in seg[seg_id_mask]: 
        length+=np.sqrt((sg['x_start']-sg['x_end'])**2+
                        (sg['y_start']-sg['y_end'])**2+
                        (sg['z_start']-sg['z_end'])**2
                       )
    return length

In [8]:
def three_momentum(pxyz):
    return float(np.sqrt(pxyz[0]**2+pxyz[1]**2+pxyz[2]**2))

In [9]:
def tuple_key_to_string(d):
    out={}
    for key in d.keys():
        string_key=""
        max_length=len(key)
        for i in range(max_length):
            if i<len(key)-1: string_key+=str(key[i])+"-"
            else: string_key+=str(key[i])
        out[string_key]=d[key]
    return out

In [10]:
def save_dict_to_json(d, name, if_tuple):
    with open(name+".json","w") as outfile:
        if if_tuple==True:
            updated_d = tuple_key_to_string(d)
            json.dump(updated_d, outfile, indent=4)
        else:
            json.dump(d, outfile, indent=4)

In [11]:
def np_array_of_array_to_flat_list(a):
    b = list(a)
    return [list(c)[0] for c in b]

In [12]:
def process_file(sim_h5):
    unique_spill = get_unique_spills(sim_h5)
    d=dict()
    for spill_id in unique_spill:
        ghdr, gstack, traj, vert, seg = get_spill_data(sim_h5, spill_id)
        
        traj_proton_mask = traj['pdgId']==2212
        proton_traj = traj[traj_proton_mask]
    
        for pt in proton_traj:
        
            # REQUIRE proton contained in 2x2 active LAr
            proton_start=pt['xyz_start']
            if fiducialized_vertex(proton_start)==False: continue
            if fiducialized_vertex(pt['xyz_end'])==False: continue
        
            # is nu vertex contained in 2x2 active LAr?
            vert_mask = vert['vertexID']==pt['vertexID']
            nu_vert = vert[vert_mask]
            vert_loc = [nu_vert['x_vert'],nu_vert['y_vert'],nu_vert['z_vert']]
            vert_loc = np_array_of_array_to_flat_list(vert_loc)
            lar_fv = 1
            if fiducialized_vertex(vert_loc)==False: lar_fv = 0
        
            # Find proton parent PDG 
            parent_mask = traj['trackID']==pt['parentID']
            parent_pdg=traj[parent_mask]['pdgId']
            if pt['parentID']==-1:
                ghdr_mask=ghdr['vertexID']==pt['vertexID']
                parent_pdg=ghdr[ghdr_mask]['nu_pdg']
        
            # Find proton grandparent PDG 
            grandparent_mask = traj['trackID']==traj[parent_mask]['parentID']
            grandparent_trackid = traj[grandparent_mask]['trackID']
            grandparent_pdg = traj[grandparent_mask]['pdgId']
            if grandparent_trackid.size>0:
                if grandparent_trackid==-1:
                    ghdr_mask=ghdr['vertexID']==pt['vertexID']
                    grandparent_pdg=ghdr[ghdr_mask]['nu_pdg']
            grandparent_pdg=list(grandparent_pdg)
            if len(grandparent_pdg)==0: grandparent_pdg=[0]
            
            if parent_pdg[0] not in [12,14,16,-12,-14,-16]:
                parent_total_energy = float(list(traj[parent_mask]['E_start'])[0])
                parent_length = float(total_edep_length(traj[parent_mask]['trackID'], traj, seg))
                parent_start_momentum = float(three_momentum(traj[parent_mask]['pxyz_start'][0]))
                parent_end_momentum = float(three_momentum(traj[parent_mask]['pxyz_end'][0]))
            else:
                parent_total_energy = float(-1) 
                parent_length = float(-1)
                parent_start_momentum = float(-1)
                parent_end_momentum = float(-1)
                
            gstack_mask = gstack['vertexID']==pt['vertexID']
            gstack_trackID = gstack[gstack_mask]['trackID']
            primary_length=[]; primary_pdg=[]
            for gid in gstack_trackID:
                primary_mask = traj['trackID']==gid
                primary_start = traj[primary_mask]['xyz_start']
                primary_end = traj[primary_mask]['xyz_end']
                p_pdg = traj[primary_mask]['pdgId']
                if len(p_pdg)==1:
                    primary_pdg.append(int(p_pdg[0]))
                    dis = np.sqrt( (primary_start[0][0]-primary_end[0][0])**2+
                                   (primary_start[0][1]-primary_end[0][1])**2+
                                   (primary_start[0][2]-primary_end[0][2])**2)
                    primary_length.append(float(dis))
                      
            p_start = proton_start.tolist()
            p_vtx = []
            for i in p_start: p_vtx.append(float(i))
            
            nu_vtx=[]
            for i in vert_loc: nu_vtx.append(float(i))
            
            d[(spill_id, pt['vertexID'], pt['trackID'])]=dict(
                lar_fv=int(lar_fv),
                
                p_vtx=p_vtx,
                nu_vtx=nu_vtx,
                
                proton_total_energy = float(pt['E_start']),
                proton_vis_energy = float(total_edep_charged_e(pt['trackID'], traj, seg)),
                proton_length = float(total_edep_length(pt['trackID'], traj, seg)),
                proton_start_momentum = float(three_momentum(pt['pxyz_start'])),
                proton_end_momentum = float(three_momentum(pt['pxyz_end'])),
                
                parent_total_energy = parent_total_energy, 
                parent_length = parent_length, 
                parent_start_momentum = parent_start_momentum, 
                parent_end_momentum = parent_end_momentum, 
                
                nu_proton_dt = float(pt['t_start']) - float(nu_vert['t_vert'][0]),
                nu_proton_distance = float(np.sqrt( (proton_start[0]-vert_loc[0])**2+
                                          (proton_start[1]-vert_loc[1])**2+
                                          (proton_start[2]-vert_loc[2])**2 )),
                
                parent_pdg=int(parent_pdg[0]),
                grandparent_pdg=int(grandparent_pdg[0]),
                primary_pdg=primary_pdg,                
                primary_length=primary_length
            )
    return d


In [13]:
for filename in glob.glob(file_dir+'*.EDEPSIM.h5'):
    file_no = filename.split(".")[-3]
    sim_h5=h5py.File(filename,'r')
    d = process_file(sim_h5)
    save_dict_to_json(d, "n_tof_"+file_no, True)

/tmp/ipykernel_1988140/189764794.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  grandparent_mask = traj['trackID']==traj[parent_mask]['parentID']
